In [1]:
import torch
_=torch.tensor([0.2135, 0.7182, 0.0782], names = ['c'])

/pytorch/c10/core/TensorImpl.h:806: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


In [0]:
img_t = torch.randn(3,8,5)
weights = torch.tensor([0.2126,0.7852,0.0702])

In [0]:
batch = torch.randn(2,3,5,5)

In [5]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([8, 5]), torch.Size([2, 5, 5]))

In [7]:
unsqueezed_weights_1 = weights.unsqueeze(-1).unsqueeze(-1)
img_weight = (img_t * unsqueezed_weights_1)
batch_weight =(batch * unsqueezed_weights_1)
img_gray_weighted = batch_weight.sum(-3)
batch_weight.shape, batch.shape, unsqueezed_weights_1.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [8]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw',img_t,weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [9]:
weights_named = torch.tensor([0.2826,0.7952,0.0752], names = ['channels'])
weights_named

tensor([0.2826, 0.7952, 0.0752], names=('channels',))

In [11]:
img_named = img_t.refine_names(...,'channels', 'rows', 'columns')
batch_named= batch.refine_names(...,'channels', 'rows', 'columns')
print("img named:", img_named.shape,img_named.names)
print("batch named", batch_named.shape, batch_named.names)

img named: torch.Size([3, 8, 5]) ('channels', 'rows', 'columns')
batch named torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [12]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape,weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [13]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([8, 5]), ('rows', 'columns'))

In [14]:
try:
    gray_named = (img_named[...,:3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [15]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names




(torch.Size([8, 5]), (None, None))